In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress

def ngram_analysis(filepath, label):
    dataset = pd.read_csv(filepath, sep='\\t', header=None, names=['Ранг', 'NGram', 'Частота'], engine='python')
    dataset['Ранг'] = pd.to_numeric(dataset['Ранг'], errors='coerce')
    dataset['Частота'] = pd.to_numeric(dataset['Частота'], errors='coerce')
    dataset = dataset.dropna()
    
    rank = dataset['Ранг']
    freq = dataset['Частота']
    
    # --- Звичайний масштаб ---
    plt.figure(figsize=(8, 6))
    plt.plot(rank, freq, 'o-', label='Дані')
    plt.title(f'Рангово-частотна залежність для {label} (звичайний масштаб)')
    plt.xlabel('Ранг')
    plt.ylabel('Частота')
    plt.grid()
    plt.legend()
    plt.show()
    
    # --- Напівлогарифмічний масштаб ---
    plt.figure(figsize=(8, 6))
    plt.semilogy(rank, freq, 'o-', label='Дані')
    plt.title(f'Рангово-частотна залежність для {label} (напівлогарифмічний масштаб)')
    plt.xlabel('Ранг')
    plt.ylabel('Частота (логарифмічний масштаб)')
    plt.grid()
    plt.legend()
    plt.show()
    
    # --- Логарифмічний масштаб ---
    log_rank = np.log10(rank)
    log_freq = np.log10(freq)
    # Лінійна апроксимація
    slope, intercept, r_value, _, _ = linregress(log_rank, log_freq)
    approx_line = slope * log_rank + intercept
    
    plt.figure(figsize=(8, 6))
    plt.plot(log_rank, log_freq, 'o', label='Дані (логарифмічний масштаб)')
    plt.plot(log_rank, approx_line, 'r-', label=f'Апроксимація: y={slope:.2f}x+{intercept:.2f}')
    plt.title(f'Рангово-частотна залежність для {label} (логарифмічний масштаб)')
    plt.xlabel('log10(Ранг)')
    plt.ylabel('log10(Частота)')
    plt.grid()
    plt.legend()
    plt.show()
    
    pearson_log = r_value
    _, _, r_value_semilog, _, _ = linregress(rank, np.log(freq))
    return {
        'n-грам': label,
        'Коефіцієнт нахилу': round(slope, 2),
        'Коефіцієнт Пірсона (логарифмічний)': round(pearson_log, 2),
        'Коефіцієнт Пірсона (напівлогарифмічний)': round(r_value_semilog, 2)
    }

files = [
    'data/1-grams.txt',
    'data/2-grams.txt',
    'data/3-grams.txt',
    'data/4-grams.txt'
]

results = []
for index, filepath in enumerate(files, start=1):
    label = f'{index}-грам'
    print(f'Аналізую {label}...')
    result = ngram_analysis(filepath, label)
    results.append(result)

df_results = pd.DataFrame(results)
print("\nТаблиця результатів:")
print(df_results.to_markdown(index=False, tablefmt="grid"))
